In [1]:
import cv2
import numpy as np
import operator
import os

In [2]:
# module level variables ##########################################################################
MIN_CONTOUR_AREA = 100

RESIZED_IMAGE_WIDTH = 20
RESIZED_IMAGE_HEIGHT = 30

In [3]:
class ContourWithData():

    # member variables ############################################################################
    npaContour = None           # contour
    boundingRect = None         # bounding rect for contour
    intRectX = 0                # bounding rect top left corner x location
    intRectY = 0                # bounding rect top left corner y location
    intRectWidth = 0            # bounding rect width
    intRectHeight = 0           # bounding rect height
    fltArea = 0.0               # area of contour

    def calculateRectTopLeftPointAndWidthAndHeight(self):               # calculate bounding rect info
        [intX, intY, intWidth, intHeight] = self.boundingRect
        self.intRectX = intX
        self.intRectY = intY
        self.intRectWidth = intWidth
        self.intRectHeight = intHeight

    def checkIfContourIsValid(self):                            # this is oversimplified, for a production grade program
        if self.fltArea < MIN_CONTOUR_AREA: return False        # much better validity checking would be necessary
        return True

In [4]:
allContoursWithData = []                # declare empty lists,
validContoursWithData = []              # we will fill these shortly

try:
    npaClassifications = np.loadtxt("classifications1.txt", np.float32)                  # read in training classifications
except:
    print("error, unable to open classifications.txt, exiting program\n")
    os.system("pause")
# end try

try:
    npaFlattenedImages = np.loadtxt("flattened_images1.txt", np.float32)                 # read in training images
except:
    print("error, unable to open flattened_images.txt, exiting program\n")
    os.system("pause")
    # end try

In [5]:
# reshape numpy array to 1d, necessary to pass to call to train
npaClassifications = npaClassifications.reshape((npaClassifications.size, 1))

In [6]:
kNearest = cv2.ml.KNearest_create()

In [7]:
kNearest.train(npaFlattenedImages, cv2.ml.ROW_SAMPLE, npaClassifications)

True

In [8]:
imgTestingNumbers = cv2.imread("test9.png")

if imgTestingNumbers is None:
    print("Error: image not found")
    os.system("pause")

In [9]:
imgGray = cv2.cvtColor(imgTestingNumbers, cv2.COLOR_BGR2GRAY)
imgBlurred = cv2.GaussianBlur(imgGray, (5,5), 0)
# cv2.imshow("gray", imgGray)
# cv2.imshow("blur", imgBlurred)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [10]:
#imgThresh = cv2.adaptiveThreshold(imgBlurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
ret, imgThresh = cv2.threshold(imgBlurred, 180, 255, cv2.THRESH_BINARY)
cv2.imshow("thresh", imgThresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
imgThreshCopy = imgThresh.copy()

In [12]:
npaContours, npaHierarchy = cv2.findContours(imgThreshCopy, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [13]:
for npaContour in npaContours:                             # for each contour
    contourWithData = ContourWithData()                                             # instantiate a contour with data object
    contourWithData.npaContour = npaContour                                         # assign contour to contour with data
    contourWithData.boundingRect = cv2.boundingRect(contourWithData.npaContour)     # get the bounding rect
    contourWithData.calculateRectTopLeftPointAndWidthAndHeight()                    # get bounding rect info
    contourWithData.fltArea = cv2.contourArea(contourWithData.npaContour)           # calculate the contour area
    allContoursWithData.append(contourWithData)                                     # add contour with data object to list of all contours with data

In [14]:
for contourWithData in allContoursWithData:                 # for all contours
    if contourWithData.checkIfContourIsValid():             # check if valid
        validContoursWithData.append(contourWithData)       # if so, append to valid contour list

In [15]:
# sort contours from left to right
validContoursWithData.sort(key = operator.attrgetter("intRectX"))

In [16]:
# final number sequence by the end of the program
strFinalString = ""

In [17]:
 for contourWithData in validContoursWithData:            # for each contour
                                                # draw a green rect around the current char
    cv2.rectangle(imgTestingNumbers,                                        # draw rectangle on original testing image
                      (contourWithData.intRectX, contourWithData.intRectY),     # upper left corner
                      (contourWithData.intRectX + contourWithData.intRectWidth, contourWithData.intRectY + contourWithData.intRectHeight),      # lower right corner
                      (0, 255, 0),              # green
                      2)                        # thickness

    imgROI = imgThresh[contourWithData.intRectY : contourWithData.intRectY + contourWithData.intRectHeight,     # crop char out of threshold image
                           contourWithData.intRectX : contourWithData.intRectX + contourWithData.intRectWidth]

    imgROIResized = cv2.resize(imgROI, (RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT))             # resize image, this will be more consistent for recognition and storage

    npaROIResized = imgROIResized.reshape((1, RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT))      # flatten image into 1d numpy array

    npaROIResized = np.float32(npaROIResized)       # convert from 1d numpy array of ints to 1d numpy array of floats

    retval, npaResults, neigh_resp, dists = kNearest.findNearest(npaROIResized, k = 1)     # call KNN function find_nearest

    strCurrentChar = str(chr(int(npaResults[0][0])))                                             # get character from results

    strFinalString = strFinalString + strCurrentChar            # append current char to full string

In [18]:
print("\n" + strFinalString + "\n")                  # show the full string

cv2.imshow("imgTestingNumbers", imgTestingNumbers)      # show input image with green boxes drawn around found digits
cv2.waitKey(0)                                          # wait for user key press

cv2.destroyAllWindows()             # remove windows from memory


4PPCAFI1Q4ITAF1TI54IKTV12I1I17IJII

